In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import os

import keras
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.models import load_model

from sklearn.metrics import f1_score

import random as rn
import datetime

In [2]:
# Reading de los modelos.

# Ruta donde están los modelos.

model_CDM = load_model(filepath = "../Versión 3 - actual/modelos/datasetCDM_2023_04_17_19_35")

model_GRD = load_model(filepath = "../Versión 3 - actual/modelos/datasetGRD_2023_04_17_20_53")

model_GRDT = load_model(filepath = "../Versión 3 - actual/modelos/datasetGRDT_2023_04_17_11_29")

model_SEV = load_model(filepath = "../Versión 3 - actual/modelos/datasetSEV_2023_04_17_22_09")

In [3]:
x = pd.read_csv("../../../Datos/Procesados/x_values.csv", dtype = int)
y_cdm = pd.read_csv("../../../Datos/Procesados/y_dummies_cdm.csv", dtype = int)
y_grdt = pd.read_csv("../../../Datos/Procesados/y_dummies_grdtipo.csv", dtype = int)
y_grd = pd.read_csv("../../../Datos/Procesados/y_dummies_grd.csv", dtype = int)
y_sev = pd.read_csv("../../../Datos/Procesados/y_dummies_sev.csv", dtype = int)

In [105]:
y = y_sev
y_str = "SEV"

In [106]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0, stratify = y)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state = 0, stratify = y_train)

In [107]:
y_prediction = model_SEV.predict(x_test)

# score = model_GRDT.evaluate(x_test, y_test, verbose = 1, batch_size = 128)

11900/11900 [==============================] - 16s 1ms/step


In [108]:
y_test_np = y_test.to_numpy(dtype = np.float32)

In [109]:
class_labels = y_test.columns

In [110]:
# class_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '10', '11', '12', '13', '14',
#        '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26',
#        '27', '29', '30', '31', '32', '33', '34', '35', '36', '37', '40',
#        '41', '42', '43', '44', '50', '51', '52', '53', '60', '61', '62', '63',
#        '64', '65', '70', '71', '72', '80', '81', '82', '83', '84']

In [111]:
class_percentages = []

for i in y_test:
    class_percentages.append(y_test[i].value_counts()[1]/len(y_test_np))

In [112]:
y_pred_classes = np.argmax(y_prediction, axis = 1, )
y_true_classes = np.argmax(y_test_np, axis = 1, keepdims = False)

In [113]:
f1_per_class = f1_score(y_true_classes, y_pred_classes, average=None)
f1_macro = f1_score(y_true_classes, y_pred_classes, average='macro')

In [123]:
def save_f1_per_class_plot_with_color_scale(f1_per_class, class_labels, class_percentages, output_folder, min_color_percentage=0.01):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    num_classes = len(class_labels)
    plt.figure(figsize=(10, 6))

    # Genera la escala de color basada en la participación de cada clase, en un rango de 0 a 15%
    # Agrega un valor mínimo al porcentaje de participación antes de escalar el color
    adjusted_percentages = np.array(class_percentages) + min_color_percentage
    color_scale = (adjusted_percentages) #+ min_color_percentage
    color_scale[color_scale > 1] = 1  # Asegura que la escala de color no supere el 100%
    colors = plt.cm.Blues(color_scale)

    plt.bar(np.arange(num_classes), f1_per_class, color=colors)
    plt.xticks(np.arange(num_classes), class_labels, rotation=90)
    plt.xlabel('Clases')
    plt.ylabel('F1-score')
    plt.title('F1-score por Clase')
    plt.savefig(os.path.join(output_folder, 'f1_per_class_color_scale_adjusted.png'), bbox_inches='tight')
    plt.close()


In [115]:
def save_f1_macro_plot(f1_macro, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    plt.figure(figsize=(6, 4))
    plt.bar(0, f1_macro, color='blue')
    plt.xticks([])
    plt.ylim(0, 1)
    plt.ylabel('F1-score')
    plt.title('F1-score Promedio (Macro)')
    plt.savefig(os.path.join(output_folder, 'f1_macro.png'), bbox_inches='tight')
    plt.close()

In [124]:
output_folder = f"../../../Visualización de datos/F1 - Score/{y_str}/"
save_f1_per_class_plot_with_color_scale(f1_per_class, class_labels, class_percentages, output_folder)
save_f1_macro_plot(f1_macro, output_folder)
